In [1]:
import numpy as np
import pandas as pd

Data = pd.read_csv('CS598PS_Project_master/friends/labels',sep='\t',header=None)
data = Data.iloc[0:135]
#labels1 = np.array(labels.iloc[1::2,0])
#labels1 = np.hstack([labels.iloc[0,0],labels1])
#print(labels1.shape)
#labels2 = pd.get_dummies(labels.iloc[::2,1])
#labels2 = np.array(labels2)
#print(labels1.shape)
#labels = np.zeros((135,9))
#labels[:,0] = labels1
#labels[:,1:9] = labels2
labels = pd.DataFrame(np.zeros((607,1)))
for i in range(135):
    first = int(data.iloc[i,2])
    second = int(data.iloc[i,3])
    labels.iloc[first:second,0]=data.iloc[i,1]
for i in range(607):
    if labels.iloc[i,0] == 'R':
        labels.iloc[i,0] = 1
    #if labels.iloc[i,0] == 'C':
     #   labels.iloc[i,0] = 2
    if labels.iloc[i,0] == 'M':
        labels.iloc[i,0] = 2
    if labels.iloc[i,0] == 'J':
        labels.iloc[i,0] = 3
    if labels.iloc[i,0] == 'S':
        labels.iloc[i,0] = 4
    else:
        labels.iloc[i,0] = 0
    #if labels.iloc[i,0] == 'P':
    #    labels.iloc[i,0] = 6
    #if labels.iloc[i,0] == 'X':
    #    labels.iloc[i,0] = 7
    #if labels.iloc[i,0] == 'Z':
    #    labels.iloc[i,0] = 8
Labels = pd.get_dummies(labels)
Labels.shape

(607, 1)

In [2]:
from scipy.io.wavfile import read as wavread
from scipy.signal import stft
import numpy as np
import matplotlib.pyplot as plt
from librosa.feature import mfcc
import librosa.display
from scipy import signal

# Load the file and convert to floating point
import scipy.io.wavfile
fs,s = scipy.io.wavfile.read('FriendsS09E01.wav')
s = s.astype( float)
print(s.shape,fs)
plt.subplot( 2, 1, 1), plt.plot( np.arange( 0, len( s))/fs, s)
plt.title( 'Input sound'), plt.xlabel( 'Time (sec)')
plt.show()
# Get the magnitude spectrogram
f, t, S = signal.stft(s,window=signal.get_window('hann',1024),noverlap=768,nperseg = 1024)
partition = len(s)/fs
slide = int(S.shape[1]//partition+1)
k = 0
data_slide = []
print(partition,slide)
while k < partition:
    left = k*slide
    right = min(S.shape[1]+1,(k+1)*slide)
    par = S[:,left:right]
    data_slide.append(par)
    k += 1
print(len(data_slide),data_slide[0].shape)

(71989248,) 48000


<Figure size 640x480 with 1 Axes>

1499.776 188
1500 (513, 188)


In [3]:
train = np.asarray(data_slide[0:607])
test = np.asarray(data_slide[607:-6])
f.shape,t.shape,s.shape,fs,len(s)/fs,S.shape,train.shape,Labels.shape,labels.shape,test.shape

((513,),
 (281209,),
 (71989248,),
 48000,
 1499.776,
 (513, 281209),
 (607, 513, 188),
 (607, 1),
 (607, 1),
 (887, 513, 188))

In [4]:
# Take as SVM with stacking
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

train_linear = np.zeros((607*188,513))
label_linear = np.zeros((607*188,1))
Label_linear = np.zeros((607*188,1))
for i in range(train.shape[0]):
    for j in range(train.shape[2]):
        train_linear[i*188:(i+1)*188,:] = train[i,:,j]
    label_linear[i*188:(i+1)*188,0] = np.repeat(labels.iloc[i,0],188).astype(int)
    Label_linear[i*188:(i+1)*188,0] = np.repeat(Labels.iloc[i,0],188).astype(int)
print(train_linear.shape,label_linear.shape)
#clf = SVC(gamma='auto')
#clf.fit(train_linear, label_linear.ravel())
#clf.get_params()
print('completed')

/Users/xiujiayang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


(114116, 513) (114116, 1)
completed


In [5]:
# Use sample to generate classifier
def get_train(file):
    fs,s = scipy.io.wavfile.read(file)
    s = s.astype(float)
    print(s.shape)
    f, t, S = signal.stft(s,window=signal.get_window('hann',1024),noverlap=768,nperseg = 1024)
    partition = len(s)/fs
    slide = int(S.shape[1]//partition+1)
    k = 0
    data_slide = []
    while k < partition:
        left = k*slide
        right = min(S.shape[1]+1,(k+1)*slide)
        par = S[:,left:right]
        data_slide.append(par)
        k += 1
    i = 1
    while data_slide[-i].shape[1] != 188:
        i += 1
    return data_slide[:-(i+1)]
train_j = get_train('J-Final-48k.wav')
train_m = get_train('M-Final-48k.wav')
train_r = get_train('R-Final-48k.wav')
train_s = get_train('S-Final-48k.wav')
labels_j = np.ones((len(train_j),1))*3
labels_m = np.ones((len(train_m),1))*2
labels_r = np.ones((len(train_r),1))
labels_s = np.ones((len(train_s),1))*4
labels_others = np.zeros((100,1))
train_data = train_j+train_m+train_r+train_s+data_slide[0:100]
train_data = np.asarray(train_data)
train_label = np.vstack([labels_j,labels_m,labels_r,labels_s,labels_others])
print(train_data.shape,train_label.shape)

(7323410,)
(7136310,)
(6377744,)
(5531383,)
(638, 513, 188) (638, 1)


In [6]:
training = np.zeros((train_data.shape[0]*train_data.shape[2],train_data.shape[1]))
labeling = np.zeros((train_data.shape[0]*train_data.shape[2],1))
j = train_data.shape[2]
print(training.shape,labeling.shape)
for i in range(train_data.shape[0]):
    for k in range(train_data.shape[2]):
        training[i*j:(i+1)*j,:] = train_data[i,:,k]
    labeling[i*j:(i+1)*j,0] = np.repeat(train_label[i,0],j).astype(int)
print(training.shape,labeling)

(119944, 513) (119944, 1)


/Users/xiujiayang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


(119944, 513) [[3.]
 [3.]
 [3.]
 ...
 [0.]
 [0.]
 [0.]]


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.multiclass import OneVsOneClassifier
import mpu.ml

labeling = labeling.astype(int).reshape(-1)
param_test1 = {
    'C':[0.01,0.05,0.1,0.5,1,5]
}
gsearch1 = GridSearchCV(estimator=LogisticRegression(solver='lbfgs',multi_class='multinomial'),
                        param_grid=param_test1,
                        scoring='f1_micro',
                        n_jobs=4,
                        iid=False,
                        cv=5)
gsearch1.fit(training, labeling)
print(gsearch1.best_params_, gsearch1.best_score_) 
clf = LogisticRegression(solver='lbfgs',multi_class='multinomial',C=0.05).fit(training,labeling)
print('train ready')
predict = clf.predict_proba(train_linear)
print(predict.shape)

/Users/xiujiayang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


{'C': 0.05} 0.2166259590462359
train ready
(114116, 5)


/Users/xiujiayang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [16]:
i = 0
slides = 188
predictions = np.array([], ).reshape(0,5)
print('slide:',slides,train_linear.shape[0])

while i*slides < train_linear.shape[0]:
    right = min((i+1)*slides,train_linear.shape[0])
    value = np.mean(predict[i*slides:right],axis=0,keepdims=1)
    predictions = np.vstack([predictions,value])
    i += 1
predictions = np.argmax(predictions,axis=1)
print(predictions.shape)
acc = accuracy_score(labels,predictions)
print(acc)

slide: 188 114116
(607,)
0.30477759472817134
